# Complete Text-to-SQL Workflow Demo

This notebook demonstrates a complete text-to-SQL workflow using all 4 specialized agents and the TaskStatusChecker:

## **Architecture Overview**
- **QueryAnalyzerAgent**: Analyzes user queries and creates query trees
- **SchemaLinkerAgent**: Links query intents to database schema  
- **SQLGeneratorAgent**: Generates SQL from linked schema
- **SQLEvaluatorAgent**: Executes and evaluates SQL results
- **TaskStatusChecker**: Determines next actions and manages tree progression

## **Key Features**
1. **Structured Memory Management**: Uses KeyValueMemory with specialized managers
2. **Query Tree Architecture**: All agents operate on nodes in a query tree
3. **Automatic SQL Execution**: SQLEvaluator automatically executes SQL against real databases
4. **Quality Assessment**: Agents evaluate SQL quality and suggest improvements
5. **Intelligent Coordination**: TaskStatusChecker manages workflow progression
6. **Complete Traceability**: Full visibility into the orchestration process

## **Workflow Process**
1. **Query Analysis**: Break down user query into intents and subqueries
2. **Schema Linking**: Map intents to relevant database tables and columns
3. **SQL Generation**: Generate SQL queries from schema-linked intents
4. **Execution & Evaluation**: Execute SQL and assess result quality
5. **Status Checking**: Determine if more work needed or task complete
6. **Iteration**: Continue until all nodes have good quality results

In [1]:
import os
import sys
import asyncio
import time
import logging
from pathlib import Path
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Check environment
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  WARNING: OPENAI_API_KEY not found in environment")
    print("   Run: source ../.env && export OPENAI_API_KEY")
else:
    print("✅ OPENAI_API_KEY found in environment")

# Set up clean logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Keep important logs but reduce noise
logging.getLogger('autogen_core.events').setLevel(logging.INFO)
logging.getLogger('autogen_core').setLevel(logging.WARNING)
logging.getLogger('autogen_agentchat').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)

print("✅ Environment and logging configured")

✅ OPENAI_API_KEY found in environment
✅ Environment and logging configured


In [2]:
# Import all required components
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager
from schema_reader import SchemaReader

# All agents + task status checker
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent
from task_status_checker import TaskStatusChecker, TaskStatusCheckerArgs

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    TableSchema, ColumnInfo, ExecutionResult
)

print("✅ All components imported successfully")
print("   - Memory managers: ✓")
print("   - 4 specialized agents: ✓") 
print("   - TaskStatusChecker: ✓")
print("   - Memory content types: ✓")

✅ All components imported successfully
   - Memory managers: ✓
   - 4 specialized agents: ✓
   - TaskStatusChecker: ✓
   - Memory content types: ✓


## 1. Initialize Memory and Managers

Set up the shared memory system and all managers that coordinate the workflow.

In [3]:
# Initialize shared memory and managers
memory = KeyValueMemory()

# Initialize all managers
task_manager = TaskContextManager(memory)
tree_manager = QueryTreeManager(memory)
schema_manager = DatabaseSchemaManager(memory)
history_manager = NodeHistoryManager(memory)

print("✅ Initialized memory and managers:")
print("   - KeyValueMemory: Central data store")
print("   - TaskContextManager: Task metadata")
print("   - QueryTreeManager: Query tree operations")
print("   - DatabaseSchemaManager: Schema information")
print("   - NodeHistoryManager: Operation history")

✅ Initialized memory and managers:
   - KeyValueMemory: Central data store
   - TaskContextManager: Task metadata
   - QueryTreeManager: Query tree operations
   - DatabaseSchemaManager: Schema information
   - NodeHistoryManager: Operation history


## 2. Load Database Schema

Load the California Schools database schema using the SchemaReader.

In [4]:
# Database configuration
data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

# Test query
test_query = "What is the highest eligible free rate for K-12 students in schools located in Alameda County?"
task_id = "workflow_demo_001"

print(f"🎯 Target Query: {test_query}")
print(f"📊 Database: {db_name}")
print("─" * 80)

# Initialize task context
await task_manager.initialize(task_id, test_query, db_name)

# Load schema using SchemaReader
schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)

# Load schema into memory
await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Get schema summary
summary = await schema_manager.get_schema_summary()
print(f"✅ Loaded database schema:")
print(f"   - Tables: {summary['table_count']}")
print(f"   - Total columns: {summary['total_columns']}")
print(f"   - Foreign keys: {summary['total_foreign_keys']}")
print(f"   - Data source: {data_path}")

2025-05-30 01:11:08,298 - TaskContextManager - INFO - Initialized task context for task workflow_demo_001


🎯 Target Query: What is the highest eligible free rate for K-12 students in schools located in Alameda County?
📊 Database: california_schools
────────────────────────────────────────────────────────────────────────────────
load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-30 01:11:20,840 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-30 01:11:20,841 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-30 01:11:20,841 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-30 01:11:20,842 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-30 01:11:20,842 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables


✅ Loaded database schema:
   - Tables: 3
   - Total columns: 89
   - Foreign keys: 2
   - Data source: /home/norman/work/text-to-sql/MAC-SQL/data/bird


## 3. Initialize All Agents

Create instances of all 4 specialized agents plus the TaskStatusChecker.

In [5]:
# LLM configuration for all agents
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

# Initialize all agents with shared memory
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)

# Initialize TaskStatusChecker (no LLM config needed - it's deterministic)
task_status_checker = TaskStatusChecker(memory)

print("✅ Initialized all agents:")
print("   - QueryAnalyzerAgent: Analyzes queries and creates trees")
print("   - SchemaLinkerAgent: Links intents to database schema")
print("   - SQLGeneratorAgent: Generates SQL from schema mappings")
print("   - SQLEvaluatorAgent: Executes and evaluates SQL")
print("   - TaskStatusChecker: Manages workflow progression")
print(f"\n🔧 LLM Configuration: {llm_config['model_name']} @ temp={llm_config['temperature']}")

2025-05-30 01:11:20,864 - QueryAnalyzerAgent - INFO - Initialized query_analyzer with model gpt-4o
2025-05-30 01:11:20,875 - SchemaLinkerAgent - INFO - Initialized schema_linker with model gpt-4o
2025-05-30 01:11:20,885 - SQLGeneratorAgent - INFO - Initialized sql_generator with model gpt-4o
2025-05-30 01:11:20,896 - SQLEvaluatorAgent - INFO - Initialized sql_evaluator with model gpt-4o


✅ Initialized all agents:
   - QueryAnalyzerAgent: Analyzes queries and creates trees
   - SchemaLinkerAgent: Links intents to database schema
   - SQLGeneratorAgent: Generates SQL from schema mappings
   - SQLEvaluatorAgent: Executes and evaluates SQL
   - TaskStatusChecker: Manages workflow progression

🔧 LLM Configuration: gpt-4o @ temp=0.1


## 4. Manual Workflow Execution

Since this is a demo notebook, we'll manually execute the workflow step by step to show how each agent works and how the TaskStatusChecker guides the process.

In [6]:
# Step 1: Query Analysis
print("🔍 STEP 1: QUERY ANALYSIS")
print("=" * 50)

# Run query analyzer
result1 = await query_analyzer.run(test_query)
print(f"✅ Query analysis complete")

# Check what was created
tree_stats = await tree_manager.get_tree_stats()
print(f"📊 Query tree stats:")
print(f"   - Total nodes: {tree_stats['total_nodes']}")
print(f"   - Max depth: {tree_stats['max_depth']}")
print(f"   - Leaf nodes: {tree_stats['leaf_count']}")

# Get root node
root_id = await tree_manager.get_root_id()
root_node = await tree_manager.get_node(root_id)
print(f"\n📄 Root node created:")
print(f"   - ID: {root_id}")
print(f"   - Intent: {root_node.intent}")
print(f"   - Status: {root_node.status}")

# Check if children were created
children = await tree_manager.get_children(root_id)
if children:
    print(f"\n👥 Child nodes created: {len(children)}")
    for i, child in enumerate(children, 1):
        print(f"   {i}. {child.nodeId}: {child.intent}")
else:
    print(f"\n👤 Single node query (no decomposition)")

2025-05-30 01:11:20,899 - QueryAnalyzerAgent - INFO - Query analyzer context prepared with schema length: 9697
2025-05-30 01:11:20,900 - QueryAnalyzerAgent - INFO - query: What is the highest eligible free rate for K-12 students in schools located in Alameda County? database: california_schools


🔍 STEP 1: QUERY ANALYSIS


2025-05-30 01:11:22,941 - autogen_core.events - INFO - {"type": "LLMCall", "messages": [{"content": "You are a query analyzer for text-to-SQL conversion. Your job is to:\n\n1. Analyze the user's query to understand their intent\n2. Identify which tables and columns are likely needed\n3. Determine query complexity\n4. For complex queries, decompose them into simpler sub-queries\n\n## Schema-Informed Analysis\n\nIf schema analysis is provided (from SchemaLinker), use it to inform your decisions:\n- **Table Selection**: Prefer tables already identified as relevant by schema analysis\n- **Column Awareness**: Consider which columns were identified as important\n- **Relationship Understanding**: Use identified foreign key relationships for decomposition\n- **Confidence Boost**: Schema analysis increases confidence in table/column choices\n\nIf no schema analysis is available, perform standard analysis based on query text and evidence.\n\n## Complexity Determination\n\n**Simple Queries** (dir

✅ Query analysis complete
📊 Query tree stats:
   - Total nodes: 0


KeyError: 'max_depth'

In [ ]:
# Step 2: Task Status Check
print("\n📍 STEP 2: TASK STATUS CHECK")
print("=" * 50)

# Check task status to see what should be done next
args = TaskStatusCheckerArgs(task="Determine next action after query analysis")
status_result = await task_status_checker.run(args)
print("TaskStatusChecker response:")
print(status_result)

# Parse the action
import re
action_match = re.search(r'ACTION:\s*(.+?)(?:\n|$)', status_result)
if action_match:
    action = action_match.group(1).strip()
    print(f"\n🎯 Recommended Action: {action}")
    
    # Extract node ID if present
    node_match = re.search(r'node:(\S+)', action) or re.search(r'NODE:\s*(\S+)', action)
    if node_match:
        recommended_node = node_match.group(1)
        print(f"📌 Target Node: {recommended_node}")
        
        # Set current node to the recommended one
        await tree_manager.set_current_node_id(recommended_node)
        print(f"✅ Current node updated to: {recommended_node}")
else:
    print("⚠️  Could not parse action from TaskStatusChecker response")

In [ ]:
# Step 3: Schema Linking
print("\n🔗 STEP 3: SCHEMA LINKING")
print("=" * 50)

# Get current node
current_node_id = await tree_manager.get_current_node_id()
current_node = await tree_manager.get_node(current_node_id)
print(f"Working on node: {current_node_id}")
print(f"Intent: {current_node.intent}")

# Run schema linker
result2 = await schema_linker.run("Link query to database schema")
print(f"✅ Schema linking complete")

# Check the mapping that was created
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.schema_linking:
    mapping = updated_node.schema_linking
    print(f"\n🗺️  Schema mapping created:")
    if mapping.get('tables'):
        print(f"   Tables: {[t['name'] for t in mapping['tables']]}")
    if mapping.get('columns'):
        print(f"   Columns: {len(mapping['columns'])} mapped")
        for col in mapping['columns'][:3]:  # Show first 3
            print(f"     - {col['table']}.{col['column']} ({col['usedFor']})")
        if len(mapping['columns']) > 3:
            print(f"     ... and {len(mapping['columns']) - 3} more")
else:
    print("⚠️  No schema mapping found")

In [ ]:
# Step 4: SQL Generation  
print("\n💻 STEP 4: SQL GENERATION")
print("=" * 50)

# Run SQL generator
result3 = await sql_generator.run("Generate SQL query")
print(f"✅ SQL generation complete")

# Check the generated SQL
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.generation and updated_node.generation.get('sql'):
    sql = updated_node.generation['sql']
    print(f"\n📝 Generated SQL:")
    print("─" * 40)
    print(sql)
    print("─" * 40)
    
    # Show any explanation if available
    if updated_node.generation.get('explanation'):
        explanation = updated_node.generation['explanation']
        print(f"\n💡 Explanation: {explanation[:200]}...")
else:
    print("⚠️  No SQL generated")

In [ ]:
# Step 5: SQL Execution & Evaluation
print("\n🚀 STEP 5: SQL EXECUTION & EVALUATION")
print("=" * 50)

# Run SQL evaluator (this will execute the SQL and evaluate results)
result4 = await sql_evaluator.run("Analyze SQL execution results")
print(f"✅ SQL execution and evaluation complete")

# Check execution results
updated_node = await tree_manager.get_node(current_node_id)
if updated_node.evaluation:
    evaluation = updated_node.evaluation
    print(f"\n📊 Execution Results:")
    
    # Show execution status
    if evaluation.get('execution_result'):
        exec_result = evaluation['execution_result']
        print(f"   Status: {exec_result.get('status', 'Unknown')}")
        print(f"   Rows returned: {exec_result.get('row_count', 0)}")
        
        if exec_result.get('data'):
            print(f"   Sample data: {exec_result['data'][:2]}...")  # First 2 rows
    
    # Show evaluation
    print(f"\n🎯 Quality Assessment:")
    print(f"   Answers intent: {evaluation.get('answers_intent', 'Unknown')}")
    print(f"   Result quality: {evaluation.get('result_quality', 'Unknown')}")
    print(f"   Confidence: {evaluation.get('confidence_score', 'N/A')}")
    
    if evaluation.get('result_summary'):
        summary = evaluation['result_summary']
        print(f"   Summary: {summary[:150]}...")
        
    # Show issues if any
    if evaluation.get('issues'):
        print(f"\n⚠️  Issues found: {len(evaluation['issues'])}")
        for issue in evaluation['issues'][:2]:  # Show first 2
            if isinstance(issue, dict):
                print(f"     - {issue.get('description', str(issue))}")
            else:
                print(f"     - {issue}")
else:
    print("⚠️  No evaluation results found")

In [ ]:
# Step 6: Final Task Status Check
print("\n🏁 STEP 6: FINAL TASK STATUS CHECK")
print("=" * 50)

# Check final task status
args = TaskStatusCheckerArgs(task="Check if workflow is complete")
final_status = await task_status_checker.run(args)
print("Final TaskStatusChecker response:")
print(final_status)

# Parse final action
action_match = re.search(r'ACTION:\s*(.+?)(?:\n|$)', final_status)
if action_match:
    final_action = action_match.group(1).strip()
    print(f"\n🏆 Final Status: {final_action}")
    
    if "TASK COMPLETE" in final_action.upper():
        print("✅ Workflow completed successfully!")
    elif "PROCESS NODE" in final_action.upper() or "RETRY" in final_action.upper():
        print("⚠️  More processing needed")
    else:
        print("❓ Unknown status")
else:
    print("⚠️  Could not parse final action")

## 5. Display Final Results

Show the complete query tree, final SQL, and execution results.

async def display_workflow_results():
    """Display comprehensive workflow results"""
    print("🏆 WORKFLOW RESULTS SUMMARY")
    print("=" * 60)
    
    # Tree statistics
    tree_stats = await tree_manager.get_tree_stats()
    print(f"📊 Query Tree Statistics:")
    print(f"   - Total nodes processed: {tree_stats['total_nodes']}")
    print(f"   - Tree depth: {tree_stats['max_depth']}")
    print(f"   - Leaf nodes: {tree_stats['leaf_count']}")
    
    # Process each node
    tree_data = await tree_manager.get_tree()
    if tree_data and "nodes" in tree_data:
        print(f"\n📋 Node Processing Summary:")
        
        for node_id, node_data in tree_data["nodes"].items():
            print(f"\n🔸 Node: {node_id}")
            print(f"   Intent: {node_data.get('intent', 'N/A')}")
            print(f"   Status: {node_data.get('status', 'N/A')}")
            
            # Show SQL if available
            if node_data.get('sql'):
                sql_preview = node_data['sql'][:100].replace('\n', ' ')
                print(f"   SQL: {sql_preview}...")
            
            # Show execution results if available  
            if node_data.get('executionResult'):
                exec_result = node_data['executionResult']
                status = exec_result.get('status', 'Unknown')
                rows = exec_result.get('rowCount', 0)
                print(f"   Execution: {status}, {rows} rows")
                
                if exec_result.get('data') and len(exec_result['data']) > 0:
                    print(f"   Result: {exec_result['data'][0]}")
    
    # Show final answer
    print(f"\n🎯 FINAL ANSWER:")
    print("─" * 40)
    
    # Find the best result
    final_node = None
    final_result = None
    
    if tree_data:
        for node_id, node_data in tree_data["nodes"].items():
            if (node_data.get('executionResult') and 
                node_data['executionResult'].get('status') == 'success' and
                node_data['executionResult'].get('data')):
                final_node = node_id
                final_result = node_data['executionResult']['data']
                break
    
    if final_result:
        print(f"✅ Query successfully answered!")
        print(f"   Node: {final_node}")
        print(f"   Result: {final_result}")
    else:
        print("❌ No successful results found")

# Display results
await display_workflow_results()

## 6. Memory Inspection

Explore the complete memory state after workflow execution.

async def display_memory_summary():
    """Display a summary of memory contents"""
    print("🧠 MEMORY CONTENTS SUMMARY")
    print("=" * 60)
    
    # Get all memory data
    memory_data = await memory.show_all(format="json")
    
    if not memory_data:
        print("Memory is empty")
        return
    
    print(f"📦 Total memory keys: {len(memory_data)}")
    
    for key, value in memory_data.items():
        print(f"\n📁 {key}")
        
        if key == "queryTree":
            tree = value["value"]
            nodes = tree.get("nodes", {})
            print(f"   - Root ID: {tree.get('rootId')}")
            print(f"   - Current Node: {tree.get('currentNodeId')}")
            print(f"   - Total Nodes: {len(nodes)}")
            
        elif key == "taskContext":
            task = value["value"]
            print(f"   - Task ID: {task.get('taskId')}")
            print(f"   - Query: {task.get('originalQuery', '')[:50]}...")
            print(f"   - Database: {task.get('databaseName')}")
            print(f"   - Status: {task.get('status')}")
            
        elif key == "databaseSchema":
            schema = value["value"]
            tables = schema.get("tables", {})
            print(f"   - Tables: {len(tables)}")
            print(f"   - Table names: {list(tables.keys())}")
            
        elif key == "nodeHistory":
            history = value["value"]
            print(f"   - Total operations: {len(history)}")
            print(f"   - Operation types: {list(set(op.get('operation') for op in history))}")
            
        elif key.endswith("_analysis"):
            analysis = value["value"]
            print(f"   - Quality: {analysis.get('result_quality', 'N/A')}")
            print(f"   - Answers intent: {analysis.get('answers_intent', 'N/A')}")
            
        else:
            # Show first few characters for other keys
            content = str(value["value"])
            preview = content[:100] + "..." if len(content) > 100 else content
            print(f"   - Content: {preview}")

# Display memory summary
await display_memory_summary()

## Summary: Complete Text-to-SQL Workflow Demonstrated

This notebook has demonstrated a complete text-to-SQL workflow using all 4 specialized agents plus the TaskStatusChecker:

### ✅ **Workflow Steps Completed**:
1. **Query Analysis**: QueryAnalyzerAgent analyzed the user query and created a query tree
2. **Schema Linking**: SchemaLinkerAgent mapped query intents to database schema
3. **SQL Generation**: SQLGeneratorAgent created SQL from schema mappings
4. **SQL Execution & Evaluation**: SQLEvaluatorAgent executed SQL and assessed quality
5. **Status Management**: TaskStatusChecker determined workflow progression

### ✅ **Key Features Demonstrated**:
- **Memory Management**: Structured data storage with specialized managers
- **Agent Coordination**: Each agent has a specific role and communicates via memory
- **Quality Assessment**: LLM-based evaluation of SQL results
- **Real Database Execution**: Actual SQL execution against California Schools database
- **Deterministic Control**: TaskStatusChecker provides rule-based workflow management

### ✅ **Architecture Benefits**:
- **Modularity**: Each agent handles one specific task
- **Traceability**: Complete history of all operations
- **Flexibility**: Can handle simple or complex queries with decomposition
- **Quality Control**: Multiple evaluation layers ensure good results
- **Extensibility**: Easy to add new agents or modify existing ones

### 🔄 **Next Steps**:
- Complex queries with multiple subqueries
- Error handling and retry mechanisms
- Performance optimization
- Integration with orchestrator agents
- Production deployment patterns

The workflow successfully converted a natural language query into accurate SQL and executed it against a real database, demonstrating the power of the multi-agent text-to-SQL architecture.